In [1]:
library(dplyr)
library(monocle3)
library(cluster)
library(ggplot2)
library(tidyverse)
library(randomNames)
library(useful)
library(Seurat)
library(patchwork)
library(cluster)
library(data.table)
library(lubridate)
library(stringr)
library(reshape2)
library(pairwise)
library(scales)
# library for coherence 
library(data.table)
library(text2vec)
library(Matrix)


Attaching package: ‘dplyr’


The following objects are masked from ‘package:stats’:

    filter, lag


The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union


Loading required package: Biobase

Loading required package: BiocGenerics

Loading required package: parallel


Attaching package: ‘BiocGenerics’


The following objects are masked from ‘package:parallel’:

    clusterApply, clusterApplyLB, clusterCall, clusterEvalQ,
    clusterExport, clusterMap, parApply, parCapply, parLapply,
    parLapplyLB, parRapply, parSapply, parSapplyLB


The following objects are masked from ‘package:dplyr’:

    combine, intersect, setdiff, union


The following objects are masked from ‘package:stats’:

    IQR, mad, sd, var, xtabs


The following objects are masked from ‘package:base’:

    Filter, Find, Map, Position, Reduce, anyDuplicated, append,
    as.data.frame, basename, cbind, colnames, dirname, do.call,
    duplicated, eval, evalq, get, grep, grepl, i

## 1. Four

In [2]:
# Load the topic modeling results
LDA = read.csv("/home/rep/scRNA-seq_clustering_to_Twitter/P2_scRNAseq_LDA_NMF/LDA_NMF/LDA_NMF_files/LDA_four_pred_contingency.csv") %>% select(-X)
NMF = read.csv("/home/rep/scRNA-seq_clustering_to_Twitter/P2_scRNAseq_LDA_NMF/LDA_NMF/LDA_NMF_files/NMF_four_pred_contingency.csv") %>% select(-X)

In [3]:
# Load the true labeling
four_after = read.csv("/home/rep/scRNA-seq_clustering_to_Twitter/P1_preprocessing/doc_metadata_stemming_four_users.csv") %>% 
    select(-X) 
four_after = four_after %>% mutate(idx = 1:nrow(four_after))

In [4]:
# load tfidf-DT matrix
tfidf_sparse_matrix = read.csv("/home/rep/scRNA-seq_clustering_to_Twitter/P2_scRNAseq_LDA_NMF/Monocle3/M3_files/doc_word_matrix_stemming_four_users_tfidf.csv") %>%
    select(-X)

#### (1) Contingency

In [5]:
# (1) LDA
df_compare = four_after %>% mutate(LDA_results = as.numeric(LDA$LDA_four_pred))
organize = df_compare  %>% dplyr::count(screen_name, LDA_results)  
contingency_table_coh = spread(organize, screen_name, n, fill = 0, convert = FALSE)
contingency_table_coh

LDA_results,PPFA,UMichFootball,breakingweather,vegancook101
<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
0,2960,97,92,48
1,165,2836,53,29
2,10,37,3045,19
3,17,17,1,3081


In [6]:
# (2) NMF
df_compare = four_after %>% mutate(NMF_results = as.numeric(NMF$NMF_four_pred))
organize = df_compare  %>% dplyr::count(screen_name, NMF_results)  
contingency_table = spread(organize, screen_name, n, fill = 0, convert = FALSE)
contingency_table

NMF_results,PPFA,UMichFootball,breakingweather,vegancook101
<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
0,5,2477,3,3
1,5,8,0,2964
2,49,139,3076,63
3,3093,363,112,147


#### (2) Top 10

In [7]:
df_compare = four_after %>% mutate(LDA_results = as.numeric(LDA$LDA_four_pred))
summary_table = data.frame(1:ncol(tfidf_sparse_matrix))

In [8]:
# (1) LDA
df_compare = four_after %>% mutate(LDA_results = as.numeric(LDA$LDA_four_pred))
summary_table = data.frame(1:ncol(tfidf_sparse_matrix))
for(j in 0:(length(unique(df_compare$LDA_results))-1)){

  cluster_j = tfidf_sparse_matrix[as.numeric(df_compare %>% filter(LDA_results == j) %>% .$idx),]
  
  words = colSums(cluster_j)/nrow(cluster_j)
  sort_by_freq = sort(words, decreasing = TRUE)
  
  summary_table = cbind(summary_table, data.frame(names(sort_by_freq)))
  }
    
summary_table = summary_table[,-1]
names(summary_table) = as.character(0:(length(unique(df_compare$LDA_results))-1))
head(summary_table, 10)

,0,1,2,3
,<chr>,<chr>,<chr>,<chr>
1,health,goblu,storm,vegan
2,care,michigan,week,recip
3,ppfa,umichfootbal,across,tofu
4,plan,game,tropic,chocol
5,parenthood,touchdown,wind,easi
6,abort,problu,rain,made
7,peopl,day,weather,salad
8,thank,big,weekend,bean
9,women,go,state,potato


In [9]:
# (1) NMF
df_compare = four_after %>% mutate(NMF_results = as.numeric(NMF$NMF_four_pred))
summary_table = data.frame(1:ncol(tfidf_sparse_matrix))
for(j in 0:(length(unique(df_compare$NMF_results))-1)){

  cluster_j = tfidf_sparse_matrix[as.numeric(df_compare %>% filter(NMF_results == j) %>% .$idx),]
  
  words = colSums(cluster_j)/nrow(cluster_j)
  sort_by_freq = sort(words, decreasing = TRUE)
  
  summary_table = cbind(summary_table, data.frame(names(sort_by_freq)))
  }
    
summary_table = summary_table[,-1]
names(summary_table) = as.character(0:(length(unique(df_compare$NMF_results))-1))
head(summary_table, 10)

,0,1,2,3
,<chr>,<chr>,<chr>,<chr>
1,goblu,vegan,storm,health
2,michigan,recip,week,care
3,problu,tofu,across,ppfa
4,touchdown,chocol,tropic,plan
5,game,easi,wind,parenthood
6,umichfootbal,made,rain,thank
7,play,salad,weather,abort
8,big,bean,weekend,peopl
9,wolverin,potato,state,women


## 2. jobs

In [10]:
# Load the topic modeling results
LDA = read.csv("/home/rep/scRNA-seq_clustering_to_Twitter/P2_scRNAseq_LDA_NMF/LDA_NMF/LDA_NMF_files/LDA_jobs_pred_contingency.csv") %>% select(-X)
NMF = read.csv("/home/rep/scRNA-seq_clustering_to_Twitter/P2_scRNAseq_LDA_NMF/LDA_NMF/LDA_NMF_files/NMF_jobs_pred_contingency.csv") %>% select(-X)

In [11]:
# Load the true labeling
jobs_after = read.csv("/home/rep/scRNA-seq_clustering_to_Twitter/P3_Proposed_workflow/PW_files/doc_metadata_stemming_jobs_hclabeled.csv") %>% 
    select(-X) 
jobs_after = jobs_after %>% mutate(idx = 1:nrow(jobs_after))

In [12]:
# load tfidf-DT matrix
tfidf_sparse_matrix = read.csv("/home/rep/scRNA-seq_clustering_to_Twitter/P3_Proposed_workflow/PW_files/doc_word_matrix_stemming_jobs_tfidf.csv") %>%
    select(-X)

#### (1) Contingency

In [13]:
# (1) LDA
df_compare = jobs_after %>% mutate(LDA_results = as.numeric(LDA$LDA_jobs_pred))
organize = df_compare  %>% dplyr::count(category, LDA_results)  
contingency_table_coh = spread(organize, category, n, fill = 0, convert = FALSE)
contingency_table_coh

LDA_results,Advertisement,Junk,News/Politics,Other,Personal
<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
0,3595,255,25,359,84
1,1930,202,48,1041,241
2,4590,265,43,1012,211
3,1281,317,513,2214,1379
4,676,1088,395,2788,3192


In [14]:
# (2) NMF
df_compare = jobs_after %>% mutate(NMF_results = as.numeric(NMF$NMF_jobs_pred))
organize = df_compare  %>% dplyr::count(category, NMF_results)  
contingency_table = spread(organize, category, n, fill = 0, convert = FALSE)
contingency_table

NMF_results,Advertisement,Junk,News/Politics,Other,Personal
<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
0,8895,485,82,898,176
1,300,157,79,643,761
2,57,83,52,551,493
3,1525,1326,777,4113,3512
4,1295,76,34,1209,165


#### (2) Top 10 words

In [15]:
df_compare = jobs_after %>% mutate(LDA_results = as.numeric(LDA$LDA_jobs_pred))
summary_table = data.frame(1:ncol(tfidf_sparse_matrix))

In [16]:
# (1) LDA
df_compare = jobs_after %>% mutate(LDA_results = as.numeric(LDA$LDA_jobs_pred))
summary_table = data.frame(1:ncol(tfidf_sparse_matrix))
for(j in 0:(length(unique(df_compare$LDA_results))-1)){

  cluster_j = tfidf_sparse_matrix[as.numeric(df_compare %>% filter(LDA_results == j) %>% .$idx),]
  
  words = colSums(cluster_j)/nrow(cluster_j)
  sort_by_freq = sort(words, decreasing = TRUE)
  
  summary_table = cbind(summary_table, data.frame(names(sort_by_freq)))
  }
    
summary_table = summary_table[,-1]
names(summary_table) = as.character(0:(length(unique(df_compare$LDA_results))-1))
head(summary_table, 10)

,0,1,2,3,4
,<chr>,<chr>,<chr>,<chr>,<chr>
1,hire,freelanc,hire,home,steve
2,health,design,manag,time,peopl
3,therapi,web,get,work,day
4,nurs,dc,sale,green,googl
5,staf,hire,engin,onlin,need
6,sunbelt,project,servic,new,data
7,soliant,pa,account,part,work
8,occup,websit,tx,creat,start
9,therapist,search,unit,stimulu,take


In [17]:
# (1) NMF
df_compare = jobs_after %>% mutate(NMF_results = as.numeric(NMF$NMF_jobs_pred))
summary_table = data.frame(1:ncol(tfidf_sparse_matrix))
for(j in 0:(length(unique(df_compare$NMF_results))-1)){

  cluster_j = tfidf_sparse_matrix[as.numeric(df_compare %>% filter(NMF_results == j) %>% .$idx),]
  
  words = colSums(cluster_j)/nrow(cluster_j)
  sort_by_freq = sort(words, decreasing = TRUE)
  
  summary_table = cbind(summary_table, data.frame(names(sort_by_freq)))
  }
    
summary_table = summary_table[,-1]
names(summary_table) = as.character(0:(length(unique(df_compare$NMF_results))-1))
head(summary_table, 10)

,0,1,2,3,4
,<chr>,<chr>,<chr>,<chr>,<chr>
1,hire,need,peopl,work,freelanc
2,manag,day,read,home,design
3,sale,worker,onlin,new,web
4,health,take,articl,steve,project
5,get,start,yet,time,develop
6,assist,far,heard,data,budget
7,therapi,anyon,variou,get,get
8,nurs,made,googl,entri,websit
9,staf,awesom,hire,green,write
